In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
from datetime import datetime
import datetime

In [2]:
game_logistics_df = pd.read_csv('total_game_logistics.csv')
away_stats_df = pd.read_csv('total_away_stats.csv')
home_stats_df = pd.read_csv('total_home_stats.csv')
box_score_df = pd.read_csv('total_box_score_df.csv')

In [3]:
all_player_stats_df = away_stats_df.append(home_stats_df)

all_player_stats_df.rename(columns = {'Unnamed: 0':'Name'}, inplace = True)

# delete all rows with team totals
non_players = all_player_stats_df[(all_player_stats_df['Name'] == 'Team Totals')].index
all_player_stats_df.drop(non_players , inplace=True)

# delete all rows with players who did not play
benched_players = all_player_stats_df[(all_player_stats_df['MP'] == 'Did Not Play')].index
all_player_stats_df.drop(benched_players , inplace=True)

# delete all rows with players who did not dress
out_players = all_player_stats_df[(all_player_stats_df['MP'] == 'Did Not Dress')].index
all_player_stats_df.drop(out_players , inplace=True)


In [4]:
all_player_stats_df = all_player_stats_df.fillna(0)
all_player_stats_df['Games'] = 1
all_player_stats_df.sort_values(by=['PTS'], ascending=False)

,Name,Team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date,Games
267,De'Aaron Fox,SAC,33:43,14.0,25.0,0.560,2.0,5.0,0.400,7.0,...,5.0,2.0,0.0,0.0,2.0,5.0,37.0,-20.0,12/19/2022,1
274,Shai Gilgeous-Alexander,OKC,37:30,16.0,23.0,0.696,2.0,3.0,0.667,3.0,...,1.0,7.0,2.0,0.0,0.0,0.0,37.0,-10.0,1/20/2023,1
57,Nikola Vučević,ORL,31:22,15.0,23.0,0.652,2.0,5.0,0.400,4.0,...,13.0,0.0,2.0,0.0,3.0,1.0,36.0,12.0,11/17/2018,1
236,De'Aaron Fox,SAC,32:26,15.0,23.0,0.652,3.0,5.0,0.600,3.0,...,5.0,4.0,1.0,1.0,3.0,1.0,36.0,5.0,10/22/2022,1
70,Tim Hardaway Jr.,NYK,35:57,9.0,20.0,0.450,5.0,10.0,0.500,9.0,...,2.0,2.0,3.0,0.0,1.0,2.0,32.0,-24.0,11/18/2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Malik Monk,SAC,13:03,0.0,4.0,0.000,0.0,2.0,0.000,0.0,...,2.0,1.0,1.0,0.0,2.0,1.0,0.0,-13.0,10/22/2022,1
180,Justin Robinson,SAC,2:52,0.0,3.0,0.000,0.0,1.0,0.000,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-3.0,12/19/2021,1
188,Daniel Theis,HOU,6:10,0.0,2.0,0.000,0.0,1.0,0.000,0.0,...,2.0,0.0,0.0,1.0,1.0,2.0,0.0,-1.0,1/14/2022,1
174,Tre Jones,SAS,10:20,0.0,0.0,0.000,0.0,0.0,0.000,0.0,...,3.0,4.0,0.0,0.0,1.0,0.0,0.0,8.0,12/19/2021,1


In [5]:
col_list = all_player_stats_df.columns.tolist()
col_list.remove('Name')
col_list.remove('MP')
col_list.remove('FG%')
col_list.remove('3P%')
col_list.remove('FT%')
col_list.remove('Date')

total_sum_list = []

for col in col_list: 
    sum_list = []
    if col == "Team":           
        players_list = all_player_stats_df["Name"].values.tolist()
        teams_list = all_player_stats_df["Team"].values.tolist()
        new_teams_list = []
        
        for i in range(len(players_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(teams_list[i])
            new_teams_list.append(temp_list)
        
        new_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
        new_teams_df = new_teams_df.drop_duplicates()

        new_teams_df = new_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
        new_teams_df = new_teams_df.set_index('Name')
    
    else:
        sum_list = all_player_stats_df.groupby('Name')[col].sum()
        total_sum_list.append(sum_list)
    
    
    
temp_player_stat_sum_df = pd.DataFrame(total_sum_list)
temp_player_stat_sum_df = temp_player_stat_sum_df.transpose()
player_stat_sum_df = pd.concat([temp_player_stat_sum_df, new_teams_df], axis=1, join="inner")

player_stat_sum_df = player_stat_sum_df.rename(columns={'3P': 'Three_P', '3PA': 'Three_PA'})

FG_list = player_stat_sum_df['FG'].tolist()
FGA_list = player_stat_sum_df['FGA'].tolist()
FGP_list = []
for i in range(len(FG_list)):
    if FGA_list[i] == 0:
        FGP_list.append(0)
    else:
        FGP = FG_list[i] / FGA_list[i]
        FGP_list.append(FGP)

player_stat_sum_df['FG%'] = FGP_list
        
Three_P_list = player_stat_sum_df['Three_P'].tolist()
Three_PA_list = player_stat_sum_df['Three_PA'].tolist()
Three_P_P_list = []
for i in range(len(Three_P_list)):
    if Three_P_list[i] == 0:
        Three_P_P_list.append(0)
    else:
        Three_P_P = Three_P_list[i] / Three_PA_list[i]
        Three_P_P_list.append(Three_P_P)

player_stat_sum_df['3P%'] = Three_P_P_list
    
FT_list = player_stat_sum_df['FT'].tolist()
FTA_list = player_stat_sum_df['FTA'].tolist()
FTP_list = []
for i in range(len(FT_list)):
    if FT_list[i] == 0:
        FTP_list.append(0)
    else:
        FTP = FT_list[i] / FTA_list[i]
        FTP_list.append(FTP)

player_stat_sum_df['FT%'] = FGP_list

player_stat_sum_df = player_stat_sum_df.rename(columns={'Three_P': '3P', 'Three_PA': '3PA'})

player_stat_sum_df = player_stat_sum_df.fillna(0)

player_stat_sum_df.loc[:, "FG%"] = player_stat_sum_df["FG%"].map('{:.3f}'.format)
player_stat_sum_df.loc[:, "3P%"] = player_stat_sum_df["3P%"].map('{:.3f}'.format)
player_stat_sum_df.loc[:, "FT%"] = player_stat_sum_df["FT%"].map('{:.3f}'.format)

FGP_Col = player_stat_sum_df.pop("FG%")
TPP_Col = player_stat_sum_df.pop("3P%")
FTP_Col = player_stat_sum_df.pop("FT%")

player_stat_sum_df.insert(6, 'FT%', FTP_Col)
player_stat_sum_df.insert(4, '3P%', TPP_Col)
player_stat_sum_df.insert(2, 'FG%', FGP_Col)

first_column = player_stat_sum_df.pop('Games')
second_column = player_stat_sum_df.pop('Team')

player_stat_sum_df.insert(0, 'Team', second_column)
player_stat_sum_df.insert(0, 'Games', first_column)


player_stat_sum_df

,Games,Team,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Name,,,,,,,,,,,,,,,,,,,,,
Aaron Gordon,3.0,ORL,23.0,41.0,0.561,6.0,13.0,0.462,5.0,8.0,...,7.0,10.0,17.0,8.0,4.0,1.0,5.0,5.0,57.0,29.0
Aaron Holiday,1.0,PHO,4.0,10.0,0.400,1.0,1.0,1.000,0.0,0.0,...,2.0,3.0,5.0,7.0,1.0,0.0,3.0,1.0,9.0,13.0
Al Horford,1.0,ATL,6.0,13.0,0.462,0.0,0.0,0.000,1.0,2.0,...,1.0,4.0,5.0,0.0,1.0,2.0,2.0,0.0,13.0,0.0
Alec Burks,2.0,UTA,11.0,19.0,0.579,2.0,4.0,0.500,10.0,15.0,...,2.0,3.0,5.0,3.0,1.0,0.0,2.0,5.0,34.0,8.0
Alex Len,5.0,ATL/SAC,11.0,18.0,0.611,0.0,2.0,0.000,1.0,6.0,...,6.0,15.0,21.0,7.0,2.0,11.0,5.0,12.0,23.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wes Iwundu,2.0,ORL,0.0,2.0,0.000,0.0,2.0,0.000,2.0,2.0,...,1.0,1.0,2.0,3.0,0.0,2.0,1.0,3.0,2.0,25.0
Wesley Johnson,1.0,LAL,7.0,14.0,0.500,1.0,3.0,0.333,0.0,0.0,...,0.0,6.0,6.0,0.0,0.0,0.0,2.0,3.0,15.0,-14.0
Xavier Munford,1.0,MEM,4.0,8.0,0.500,1.0,1.0,1.000,1.0,2.0,...,0.0,2.0,2.0,1.0,2.0,0.0,3.0,3.0,10.0,-5.0


In [12]:
player_stat_sum_df.sort_values(by=['3P%'], ascending=False).head(50)

,Games,Team,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Name,,,,,,,,,,,,,,,,,,,,,
D.J. Augustin,3.0,HOU/ORL,13.0,17.0,0.765,7.0,7.0,1.000,12.0,14.0,...,0.0,6.0,6.0,14.0,0.0,0.0,7.0,7.0,45.0,24.0
Noah Vonleh,1.0,NYK,2.0,5.0,0.400,1.0,1.0,1.000,2.0,2.0,...,2.0,7.0,9.0,1.0,0.0,1.0,2.0,4.0,7.0,-4.0
Reggie Jackson,1.0,LAC,4.0,5.0,0.800,3.0,3.0,1.000,3.0,3.0,...,0.0,2.0,2.0,5.0,1.0,0.0,3.0,2.0,14.0,0.0
Jahmi'us Ramsey,2.0,SAC,3.0,4.0,0.750,1.0,1.0,1.000,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,7.0,6.0
Xavier Munford,1.0,MEM,4.0,8.0,0.500,1.0,1.0,1.000,1.0,2.0,...,0.0,2.0,2.0,1.0,2.0,0.0,3.0,3.0,10.0,-5.0
Allonzo Trier,1.0,NYK,2.0,5.0,0.400,1.0,1.0,1.000,2.0,2.0,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,7.0,-11.0
Aaron Holiday,1.0,PHO,4.0,10.0,0.400,1.0,1.0,1.000,0.0,0.0,...,2.0,3.0,5.0,7.0,1.0,0.0,3.0,1.0,9.0,13.0
Pascal Siakam,1.0,TOR,10.0,12.0,0.833,3.0,3.0,1.000,9.0,10.0,...,0.0,8.0,8.0,1.0,0.0,2.0,4.0,3.0,32.0,19.0
Nicolas Batum,1.0,LAC,3.0,3.0,1.000,1.0,1.0,1.000,0.0,0.0,...,1.0,4.0,5.0,3.0,2.0,1.0,1.0,5.0,7.0,-3.0


In [7]:
col_list = all_player_stats_df.columns.tolist()
col_list.remove('Name')
col_list.remove('MP')
col_list.remove('FG%')
col_list.remove('3P%')
col_list.remove('FT%')
col_list.remove('Date')

total_average_list = []

for col in col_list: 
    average_list = []
    teams_list = []
    
    if col == "Games":
        average_list = all_player_stats_df.groupby('Name')[col].sum()
        total_average_list.append(average_list)
        
    elif col == "Team":           
        players_list = all_player_stats_df["Name"].values.tolist()
        teams_list = all_player_stats_df["Team"].values.tolist()
        new_teams_list = []
        
        for i in range(len(players_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(teams_list[i])
            new_teams_list.append(temp_list)
        
        new_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
        new_teams_df = new_teams_df.drop_duplicates()

        new_teams_df = new_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
        new_teams_df = new_teams_df.set_index('Name')

    else:
        average_list = all_player_stats_df.groupby('Name')[col].mean()
        total_average_list.append(average_list)
    
temp_player_stat_average_df = pd.DataFrame(total_average_list)
temp_player_stat_average_df = temp_player_stat_average_df.transpose()
player_stat_average_df = pd.concat([temp_player_stat_average_df, new_teams_df], axis=1, join="inner")

player_stat_average_df = player_stat_average_df.rename(columns={'3P': 'Three_P', '3PA': 'Three_PA'})

FG_list2 = player_stat_average_df['FG'].tolist()
FGA_list2 = player_stat_average_df['FGA'].tolist()
FGP_list2 = []
for i in range(len(FG_list2)):
    if FGA_list2[i] == 0:
        FGP_list2.append(0)
    else:
        FGP2 = FG_list2[i] / FGA_list2[i]
        FGP_list2.append(FGP2)

player_stat_average_df['FG%'] = FGP_list2
        
Three_P_list2 = player_stat_average_df['Three_P'].tolist()
Three_PA_list2 = player_stat_average_df['Three_PA'].tolist()
Three_P_P_list2 = []
for i in range(len(Three_P_list2)):
    if Three_P_list2[i] == 0:
        Three_P_P_list2.append(0)
    else:
        Three_P_P2 = Three_P_list2[i] / Three_PA_list2[i]
        Three_P_P_list2.append(Three_P_P2)

player_stat_average_df['3P%'] = Three_P_P_list2
    
FT_list2 = player_stat_average_df['FT'].tolist()
FTA_list2 = player_stat_average_df['FTA'].tolist()
FTP_list2 = []
for i in range(len(FT_list2)):
    if FT_list2[i] == 0:
        FTP_list2.append(0)
    else:
        FTP2 = FT_list2[i] / FTA_list2[i]
        FTP_list2.append(FTP2)

player_stat_average_df['FT%'] = FGP_list2
      
player_stat_average_df = player_stat_average_df.rename(columns={'Three_P': '3P', 'Three_PA': '3PA'})

player_stat_average_df = player_stat_average_df.fillna(0)

player_stat_average_df.loc[:, "FG%"] = player_stat_average_df["FG%"].map('{:.3f}'.format)
player_stat_average_df.loc[:, "3P%"] = player_stat_average_df["3P%"].map('{:.3f}'.format)
player_stat_average_df.loc[:, "FT%"] = player_stat_average_df["FT%"].map('{:.3f}'.format)

FGP_Col = player_stat_average_df.pop("FG%")
TPP_Col = player_stat_average_df.pop("3P%")
FTP_Col = player_stat_average_df.pop("FT%")

player_stat_average_df.insert(6, 'FT%', FTP_Col)
player_stat_average_df.insert(4, '3P%', TPP_Col)
player_stat_average_df.insert(2, 'FG%', FGP_Col)

player_stat_average_df = player_stat_average_df.round(2)

first_column2 = player_stat_average_df.pop('Games')
second_column2 = player_stat_average_df.pop('Team')

player_stat_average_df.insert(0, 'Team', second_column2)
player_stat_average_df.insert(0, 'Games', first_column2)

player_stat_average_df

,Games,Team,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Name,,,,,,,,,,,,,,,,,,,,,
Aaron Gordon,3.0,ORL,7.67,13.67,0.561,2.0,4.33,0.462,1.67,2.67,...,2.33,3.33,5.67,2.67,1.33,0.33,1.67,1.67,19.0,9.67
Aaron Holiday,1.0,PHO,4.00,10.00,0.400,1.0,1.00,1.000,0.00,0.00,...,2.00,3.00,5.00,7.00,1.00,0.00,3.00,1.00,9.0,13.00
Al Horford,1.0,ATL,6.00,13.00,0.462,0.0,0.00,0.000,1.00,2.00,...,1.00,4.00,5.00,0.00,1.00,2.00,2.00,0.00,13.0,0.00
Alec Burks,2.0,UTA,5.50,9.50,0.579,1.0,2.00,0.500,5.00,7.50,...,1.00,1.50,2.50,1.50,0.50,0.00,1.00,2.50,17.0,4.00
Alex Len,5.0,ATL/SAC,2.20,3.60,0.611,0.0,0.40,0.000,0.20,1.20,...,1.20,3.00,4.20,1.40,0.40,2.20,1.00,2.40,4.6,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wes Iwundu,2.0,ORL,0.00,1.00,0.000,0.0,1.00,0.000,1.00,1.00,...,0.50,0.50,1.00,1.50,0.00,1.00,0.50,1.50,1.0,12.50
Wesley Johnson,1.0,LAL,7.00,14.00,0.500,1.0,3.00,0.333,0.00,0.00,...,0.00,6.00,6.00,0.00,0.00,0.00,2.00,3.00,15.0,-14.00
Xavier Munford,1.0,MEM,4.00,8.00,0.500,1.0,1.00,1.000,1.00,2.00,...,0.00,2.00,2.00,1.00,2.00,0.00,3.00,3.00,10.0,-5.00


In [8]:
player_stat_average_df.sort_values(by=['PTS'], ascending=False).head(50)

,Games,Team,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Name,,,,,,,,,,,,,,,,,,,,,
Shai Gilgeous-Alexander,1.0,OKC,16.00,23.00,0.696,2.00,3.00,0.667,3.00,4.00,...,1.00,0.00,1.00,7.00,2.00,0.00,0.00,0.00,37.00,-10.00
Tim Hardaway Jr.,1.0,NYK,9.00,20.00,0.450,5.00,10.00,0.500,9.00,10.00,...,1.00,1.00,2.00,2.00,3.00,0.00,1.00,2.00,32.00,-24.00
Jayson Tatum,1.0,BOS,12.00,16.00,0.750,7.00,10.00,0.700,1.00,1.00,...,0.00,5.00,5.00,4.00,0.00,0.00,2.00,1.00,32.00,40.00
Tyrese Maxey,1.0,PHI,12.00,29.00,0.414,2.00,8.00,0.250,6.00,6.00,...,0.00,3.00,3.00,6.00,0.00,0.00,2.00,2.00,32.00,-4.00
Pascal Siakam,1.0,TOR,10.00,12.00,0.833,3.00,3.00,1.000,9.00,10.00,...,0.00,8.00,8.00,1.00,0.00,2.00,4.00,3.00,32.00,19.00
Kelly Oubre Jr.,1.0,CHO,12.00,25.00,0.480,3.00,8.00,0.375,4.00,6.00,...,1.00,8.00,9.00,1.00,1.00,1.00,4.00,2.00,31.00,1.00
Jaylen Brown,1.0,BOS,11.00,18.00,0.611,3.00,5.00,0.600,5.00,8.00,...,1.00,2.00,3.00,0.00,2.00,0.00,2.00,2.00,30.00,-5.00
Trae Young,1.0,ATL,12.00,22.00,0.545,3.00,10.00,0.300,3.00,3.00,...,2.00,0.00,2.00,12.00,5.00,0.00,5.00,0.00,30.00,-6.00
Nikola Vučević,3.0,ORL,11.67,19.00,0.614,1.33,2.67,0.500,5.00,6.00,...,3.67,8.00,11.67,4.00,1.33,1.00,3.00,2.00,29.67,12.67


In [9]:
# val = all_player_stats_df['MP'].values[0]
# val2 = all_player_stats_df['MP'].values[1]

# datetime_object = datetime.datetime.strptime(val, '%M:%S').time()
# datetime_object2 = datetime.datetime.strptime(val2, '%M:%S').time()

# timeList = [val, val2]

# mysum = datetime.timedelta()
# for i in timeList:
#     (m, s) = i.split(':')
#     d = datetime.timedelta(minutes=int(m), seconds=int(s))
#     mysum += d
# print(str(mysum))

# time_list = all_player_stats_df['MP']
# time_sum_list = []
# for i in range(len(time_list)):
#     val = all_player_stats_df['MP'].values[i]
#     datetime_object = datetime.datetime.strptime(val, '%M:%S').time()


1:03:35
